In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.model_selection import train_test_split

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
train = pd.read_csv('./train.csv')

# pd.read_csv('../input/feedback-prize-english-language-learning/train.csv')
test = pd.read_csv('./test.csv')
# pd.read_csv('../input/feedback-prize-english-language-learning/test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')
# pd.read_csv('../input/feedback-prize-english-language-learning/sample_submission.csv')

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

/m/home/home7/77/spilsbt1/unix/english_feedback/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
test.head()

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [5]:
# add additional test data because we only have 3 test data point
training_data, additional_test  = train_test_split(train, train_size=0.95, random_state=1)

In [6]:
train_df, val_df = train_test_split(training_data, train_size=0.8, random_state=1)

In [7]:
sample_submission.head()

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


In [8]:
train_x, train_y = np.array(train_df["full_text"]), np.array(train_df[["cohesion", "syntax", "vocabulary", "phraseology", "grammar", "conventions"]])
val_x, val_y = np.array(val_df["full_text"]), np.array(val_df[["cohesion", "syntax", "vocabulary", "phraseology", "grammar", "conventions"]])


In [9]:
test_x = np.array(test["full_text"])
test_y = np.array(sample_submission[["cohesion", "syntax", "vocabulary", "phraseology", "grammar", "conventions"]])

In [10]:
additional_test_x, additional_test_y = np.array(additional_test["full_text"]), np.array(additional_test[["cohesion", "syntax", "vocabulary", "phraseology", "grammar", "conventions"]])


In [11]:
train_y

array([[3.5, 3.5, 3. , 3.5, 3.5, 3. ],
       [3.5, 3.5, 4. , 4. , 3.5, 3.5],
       [3. , 3. , 4. , 3.5, 4.5, 4. ],
       ...,
       [4. , 3.5, 4.5, 4.5, 3.5, 4. ],
       [2.5, 2.5, 2. , 2.5, 2. , 2.5],
       [2.5, 3. , 3. , 3. , 3.5, 3. ]])

In [12]:
import pdb

In [13]:
def get_token_and_masks(sentence_arr):
    
    tokens_arr = []
    tokens_type_ids_arr = []
    attention_mask = []
    for item in sentence_arr:
#         pdb.set_trace()
        tokens = tokenizer(item, padding="max_length", truncation=True, max_length=512)
        if len(tokens["input_ids"]) != 512:
            print("wtf",len(tokens["input_ids"]), tokens)
        tokens_arr.append(np.array(tokens["input_ids"]))
        tokens_type_ids_arr.append(np.array(tokens["token_type_ids"]))
        attention_mask.append(np.array(tokens["attention_mask"]))
    return np.array(tokens_arr), np.array(tokens_type_ids_arr), np.array(attention_mask)

In [14]:
train_tokens_arr, train_token_type_ids_arr, train_attention_mask = get_token_and_masks(train_x)
val_tokens_arr, val_token_type_ids_arr, val_attention_mask = get_token_and_masks(val_x)


In [15]:
test_tokens_arr, test_token_type_ids_arr, test_attention_mask = get_token_and_masks(test_x)

In [16]:
a_test_tokens_arr, a_test_token_type_ids_arr, a_test_attention_mask = get_token_and_masks(additional_test_x)

### DataLoader

In [17]:
train_dataset = list(zip(train_tokens_arr, train_token_type_ids_arr, train_attention_mask, train_y))
val_dataset = list(zip(val_tokens_arr, train_token_type_ids_arr, val_attention_mask, val_y))


In [18]:
test_dataset = list(zip(test_tokens_arr, test_token_type_ids_arr, test_attention_mask))

In [19]:
a_test_dataset = list(zip(a_test_tokens_arr, a_test_token_type_ids_arr, a_test_attention_mask))

In [20]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4, drop_last=True)
val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=4, drop_last=True)

In [21]:
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=4, drop_last=False)

In [22]:
a_test_dataset

[(array([  101,  1752, 25263,  1132,  1593,  4763,  1106,  1849,   119,
           146, 23423,  1272,  1165,  1128,  1440,  1120,   170,  1825,
          1111,  1750,  1190,   127,  3071,  1128,  1341,  1128,  1221,
          1115,  1825,   119,  5226,  1128,  1138,   170,  3662,  1891,
          1114,  1115,  1825,  1120,  1655,  1111,  1995, 22532, 20311,
           119,  2353,  8351,  1209,  1849,   119,  1438,   178, 23423,
          1115,  1148, 25263,  1110,  1593,  4763,  1106,  1849,   119,
          1109,  2255,  1725,  1110,  1115,  1103,  1236,   170,  1825,
          2736,   117,  5935,   117,  1105,  1147,  4909,  4196,  1132,
          1579,  1115,  1849,  1240,  2006,  1553,  1104,  2458,  1104,
          1115,  1825,   119,  1622,  1240,  1148,  8351,   117,  1165,
          1128,  1440,  1120,  1172,   119,  1438,  1103,  1236,   170,
          1825,  2736,  1121,  1240,  1148,  8351,  2762,  1204,  1579,
          1150,  1152,  1541,  1132,   119,  2353,  8310,  1113,

In [23]:
additional_test_dataloader = DataLoader(a_test_dataset, shuffle=False, batch_size=4, drop_last=False)

In [24]:
# tokenizer(train_x[0], padding=True, truncation=True, max_length=128)["input_ids"]

In [25]:
# for token, ids, mask, label in train_dataloader:
#     print(token, ids, mask, label)
    

### Model

In [26]:
import torch.nn as nn

In [27]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


In [28]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [29]:
class ModelWrapper(nn.Module):
    def __init__(self, bert_module, n_outputs, scale):
        super().__init__()
        self.bert_module = bert_module
        self.linear = nn.Linear(768, 6)
        self.scale = scale
    
    def forward(self, input_ids, token_type_ids, attention_mask):
        pooler_output = self.bert_module(
            input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
        ).pooler_output
        return self.linear(pooler_output).sigmoid() * self.scale

In [30]:
wrapper = ModelWrapper(model.bert, 6, 5.5)

### Optimizer and learning rate scheduler

In [31]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

In [32]:
from transformers import get_scheduler

num_epochs = 10
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=1000, num_training_steps=num_training_steps
)

2022-10-30 16:33:25.806913: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-30 16:33:25.953352: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-30 16:33:25.953395: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-30 16:33:25.973565: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-30 16:33:28.090331: W tensorflow/stream_executor/pla

In [33]:
wrapper.to(device)

ModelWrapper(
  (bert_module): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [34]:
from tqdm.auto import tqdm

def train():
    progress_bar = tqdm(range(num_training_steps))
    mse_loss = nn.MSELoss()

    wrapper.train()
    
    try:
        for epoch in range(num_epochs):
            for tokens, ids, masks, labels in train_dataloader:
#                 print("epoch", epoch)
                tokens = tokens.to(device)
                ids = ids.to(device)
                masks = masks.to(device)
                labels = labels.to(device)
                outputs = wrapper(tokens, ids, masks)
        #     for batch in train_dataloader:
        # #         batch = {k: v.to(device) for k, v in batch.items()}
        #         outputs = wrapper(
        #             input_ids=batch['input_ids'],
        #             token_type_ids=batch['token_type_ids'],
        #             attention_mask=batch['attention_mask']
        #         )

        #     F.mse_loss(labels, outputs)
                loss = mse_loss(outputs, labels.float())
                loss.backward()

                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress_bar.update(1)
                progress_bar.set_postfix({"loss": loss.item()})
    except KeyboardInterrupt:
        print("interrupt")

    torch.save(wrapper, "saved_model.pt")
    return wrapper

wrapper = train()

100%|██████████████████████████| 7430/7430 [39:06<00:00,  3.17it/s, loss=0.0195]


In [35]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [36]:
model = torch.load("saved_model.pt")
model.to(device)
model.eval()

ModelWrapper(
  (bert_module): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [37]:
for tokens, ids, masks in test_dataloader:
    tokens = tokens.to(device)
    ids = ids.to(device)
    masks = masks.to(device)
    preds = wrapper(tokens, ids, masks)

In [38]:
preds

tensor([[2.8260, 2.7912, 3.3707, 3.2626, 2.4979, 2.5551],
        [2.4234, 2.5580, 2.8198, 2.6633, 2.1876, 2.9309],
        [3.9733, 3.2418, 3.8925, 3.4019, 3.3861, 3.3349]], device='cuda:0',
       grad_fn=<MulBackward0>)

In [39]:
sample_submission.head()

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


In [40]:
def evaluation(model, dataloader):
    with torch.inference_mode():
        evaluation_prediction_arr = []
        for tokens, ids, masks in dataloader:
            tokens = tokens.to(device)
            ids = ids.to(device)
            masks = masks.to(device)
            a_preds = model(tokens, ids, masks).cpu().detach().numpy()
            pred_transform = np.around(a_preds*2)/2
            evaluation_prediction_arr.append(pred_transform)
        return np.concatenate(evaluation_prediction_arr)

In [41]:
from sklearn.metrics import mean_squared_error


In [42]:
additional_test_y.shape

(196, 6)

In [43]:
a = evaluation(wrapper, additional_test_dataloader)

In [44]:
additional_test_y

array([[4. , 3.5, 3.5, 4. , 3.5, 4. ],
       [3. , 3. , 3.5, 3. , 2. , 2. ],
       [2.5, 3.5, 3.5, 3.5, 2.5, 3. ],
       ...,
       [3. , 2.5, 2.5, 2. , 2.5, 2. ],
       [3. , 2. , 3. , 2.5, 2. , 3. ],
       [3.5, 3.5, 3. , 3.5, 3. , 3. ]])

In [45]:
mean_squared_error(additional_test_y, a)

0.275297619047619

In [46]:
model_rmsw = torch.load("saved_model_rmsw.pt")
model_rmsw.to(device)
model_rmsw.eval()

ModelWrapper(
  (bert_module): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [47]:
preds_orig_rmsw = evaluation(model_rmsw, additional_test_dataloader)

In [48]:
mean_squared_error(additional_test_y, preds_orig_rmsw)

0.413265306122449

In [49]:
! nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sun Oct 30 17:12:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:B3:00.0 Off |                  N/A |
| 34